### Will Yadier Molina be inducted into the MLB Hall of Fame?
This is a machine learning exercise with the objective of answering this controversial question. Will Yadier Molina be inducted into the hall of fame? This is a second version of this exercise. The improvements to this algorithm relative to the previous is the evaluation of different machie learning models and not just two. The best model is selected and then fed with Yadier Molina's data.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import pandas_profiling

# Import libraries for model validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

# Import machine learning model libraries
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost

### Importing and checking the data

In [2]:
# Import the data frame with the catchers information. This dataframe was gathered in the past algorithm
stats_df = pd.read_csv('molina_stats_df.csv', index_col=0)
stats_df.head(2)

,playerID,nameFirst,nameLast,birthYear,birthCountry,G_c,G_all,AB,R,RBI,H,2B,3B,HR,AVG,PB,SB,inducted
0,allisdo01,Doug,Allison,1846.0,USA,277.0,316.0,1407.0,236.0,139.0,382.0,44.0,10.0,2.0,0.271500,166.0,0.0,False
1,alomasa02,Sandy,Alomar,1966.0,P.R.,1324.0,1377.0,4530.0,520.0,588.0,1236.0,249.0,10.0,112.0,0.272848,71.0,758.0,False


In [3]:
# Collect the Yadier Molina's Data. This data has the X format, which is different than the above format for all the other catchers
# This is a manual input with the numbers updated to sep-21-2022
molinaDict = [{'G_c':2180 , # Games as catcher
              'G_all':2221, # Games played
              'AB':7806,    # At-bat
              'R':777,      # Runs scored
              'RBI':1020,   # Runs Batted In
              'H':2167,     # Hits
              '2B':408,     # Doubles
              '3B':7,       # Triples
              'HR':176,     # Home Runs
              'AVG':278,    # Batting Average
              'PB':96,      # Pastball
              'SB':562}]    # Defensive Stolen Bases
molina = pd.DataFrame(molinaDict)
molina

,G_c,G_all,AB,R,RBI,H,2B,3B,HR,AVG,PB,SB
0,2180,2221,7806,777,1020,2167,408,7,176,278,96,562


In [4]:
# Checking the datatypes
stats_df.dtypes

playerID         object
nameFirst        object
nameLast         object
birthYear       float64
birthCountry     object
G_c             float64
G_all           float64
AB              float64
R               float64
RBI             float64
H               float64
2B              float64
3B              float64
HR              float64
AVG             float64
PB              float64
SB              float64
inducted           bool
dtype: object

In [5]:
# Profile report on the stats dataframe. This code takes 1 minute to run. Uncomment if needed
pandas_profiling.ProfileReport(stats_df)

In [6]:
# Check descriptive statistics values for the stats dataframe
stats_df.describe()

,birthYear,G_c,G_all,AB,R,RBI,H,2B,3B,HR,AVG,PB,SB
count,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,1919.327586,1093.327586,1384.474138,4546.267241,567.543103,615.724138,1225.948276,212.396552,34.163793,103.491379,0.263997,103.974138,433.215517
std,36.110164,529.067001,555.743817,2070.563840,340.851756,348.163544,612.252503,118.757545,27.013988,104.234476,0.025724,100.738789,427.542763
min,1846.000000,67.000000,120.000000,343.000000,38.000000,39.000000,82.000000,8.000000,3.000000,2.000000,0.192708,6.000000,0.000000
25%,1889.750000,699.000000,939.500000,2892.000000,313.250000,334.000000,733.250000,120.500000,15.750000,20.000000,0.244849,56.000000,0.000000
50%,1922.000000,1113.000000,1435.000000,4493.000000,528.500000,586.500000,1209.500000,204.500000,27.500000,67.000000,0.264832,75.500000,310.500000
75%,1949.000000,1456.500000,1776.000000,5864.000000,712.750000,815.500000,1582.000000,272.250000,47.250000,174.500000,0.282194,121.500000,753.500000
max,1974.000000,2427.000000,2850.000000,10876.000000,1844.000000,1430.000000,3060.000000,668.000000,178.000000,427.000000,0.319598,639.000000,1586.000000


In [7]:
# Check for null values
# stats_df.isnull().sum() provides the null by feature
# stats_df.isnull().sum().sum() provides the total of null values
stats_df.isnull().sum().sum()

0

In [8]:
# See all of the different values for a particular column
stats_df['3B'].value_counts()

10.0     5
12.0     5
27.0     4
34.0     4
6.0      4
        ..
45.0     1
178.0    1
40.0     1
72.0     1
76.0     1
Name: 3B, Length: 61, dtype: int64

In [9]:
# Normalize data. This is a procedure which gets z value for all the cells.
# Z value is a descrptive statistics value which indicates how many standard deviations a value is away relative to the mean
scaler = preprocessing.StandardScaler()
stats_df_normalized = scaler.fit_transform(stats_df.drop(['playerID', 'nameFirst', 'nameLast', 'birthYear','birthCountry', 'inducted'], axis=1))

In [28]:
# Split the data into training and test data and create a baseline model using Dummy Classifier
X_train, X_test, y_train, y_test = train_test_split(stats_df_normalized, stats_df['inducted'], test_size=0.3, random_state=42)
bm = DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_test, y_test)

0.8285714285714286

In [11]:
# Create a models list. It will be used in the next steps
models_list = [
    DummyClassifier,
    LogisticRegression,
    DecisionTreeClassifier,
    KNeighborsClassifier,
    GaussianNB,
    SVC,
    RandomForestClassifier,
    xgboost.XGBClassifier
]

In [22]:
# Observe the different scores for the different models for normalized data
X = stats_df_normalized
y = stats_df['inducted'].values

for model in models_list:
    cls = model()
    kfold = KFold(n_splits=10, random_state=42, shuffle=True)
    s = cross_val_score(cls, X, y, cv=kfold)
    print(
        f"{model.__name__:22} AUC: {s.mean():.3f} STD: {s.std():.2f}"
    )

DummyClassifier        AUC: 0.791 STD: 0.09
LogisticRegression     AUC: 0.834 STD: 0.10
DecisionTreeClassifier AUC: 0.748 STD: 0.06
KNeighborsClassifier   AUC: 0.844 STD: 0.08
GaussianNB             AUC: 0.801 STD: 0.09
SVC                    AUC: 0.858 STD: 0.13
RandomForestClassifier AUC: 0.852 STD: 0.08
XGBClassifier          AUC: 0.819 STD: 0.08


In [25]:
# With the best model predict for Yadier Molina
best_model = SVC()
best_model.fit(X,y)
best_model.predict(molina)

C:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


array([False])

In [26]:
# Observe the different predictions for the available models
for model in models_list:
    print(
        f'{model.__name__:22} Prediction: {model().fit(X,y).predict(molina)}'
    )

DummyClassifier        Prediction: [False]
LogisticRegression     Prediction: [ True]
DecisionTreeClassifier Prediction: [ True]
KNeighborsClassifier   Prediction: [ True]
GaussianNB             Prediction: [ True]
SVC                    Prediction: [False]


C:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
C:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
C:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(
C:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


RandomForestClassifier Prediction: [ True]
XGBClassifier          Prediction: [1]


C:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
